In [1]:
import sagemaker

sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name

bucket = sagemaker_session.default_bucket()
prefix = "sagemaker/DEMO-pytorch-hachathon_test3"

role = sagemaker.get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
inputs = sagemaker_session.upload_data(path="../data", bucket=bucket, key_prefix=prefix)
print("input spec (in this case, just an S3 path): {}".format(inputs))

input spec (in this case, just an S3 path): s3://sagemaker-us-east-1-485578841830/sagemaker/DEMO-pytorch-hachathon_test2


In [3]:
import boto3

# Create a boto3 S3 client
s3_client = boto3.client('s3')

# Define the bucket and prefix based on the previous upload
bucket = sagemaker_session.default_bucket()

# List objects within the specified bucket and prefix
response = s3_client.list_objects_v2(Bucket=bucket, Prefix=prefix)

# Iterate over the objects and print the file names
if 'Contents' in response:
    for obj in response['Contents']:
        file_name = obj['Key']
        print(file_name)
else:
    print("No files found.")


sagemaker/DEMO-pytorch-hachathon_test2/fsc22/audio/25_12526.wav
sagemaker/DEMO-pytorch-hachathon_test2/fsc22/audio/25_12527.wav
sagemaker/DEMO-pytorch-hachathon_test2/fsc22/audio/25_12528.wav
sagemaker/DEMO-pytorch-hachathon_test2/fsc22/audio/25_12529.wav
sagemaker/DEMO-pytorch-hachathon_test2/fsc22/audio/25_12530.wav
sagemaker/DEMO-pytorch-hachathon_test2/fsc22/audio/25_12531.wav
sagemaker/DEMO-pytorch-hachathon_test2/fsc22/audio/25_12532.wav
sagemaker/DEMO-pytorch-hachathon_test2/fsc22/audio/25_12533.wav
sagemaker/DEMO-pytorch-hachathon_test2/fsc22/audio/25_12534.wav
sagemaker/DEMO-pytorch-hachathon_test2/fsc22/audio/25_12535.wav
sagemaker/DEMO-pytorch-hachathon_test2/fsc22/audio/25_12536.wav
sagemaker/DEMO-pytorch-hachathon_test2/fsc22/audio/25_12537.wav
sagemaker/DEMO-pytorch-hachathon_test2/fsc22/audio/25_12538.wav
sagemaker/DEMO-pytorch-hachathon_test2/fsc22/audio/25_12539.wav
sagemaker/DEMO-pytorch-hachathon_test2/fsc22/audio/25_12540.wav
sagemaker/DEMO-pytorch-hachathon_test2/f

## Train

In [4]:
# from sagemaker.pytorch import PyTorch

# estimator = PyTorch(
#     entry_point="audio_resnet.py",
#     role=role,
#     py_version="py38",
#     framework_version="1.11.0",
#     instance_count=2,
#     instance_type="ml.c5.2xlarge",
#     hyperparameters={"epochs": 10, "backend": "gloo"},
# )

In [5]:
# estimator.fit({"training": inputs})

In [11]:
import sagemaker
from sagemaker.pytorch import PyTorch

# Set up the SageMaker session
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# Define the S3 bucket and prefix where the training data is stored
bucket = sagemaker_session.default_bucket()
prefix = "sagemaker/DEMO-pytorch-hachathon"

# Define the path to the training script
entry_point_script = "audio_resnet.py"

# Define hyperparameters
hyperparameters = {
    "epochs": 10,
    "batch_size": 32,
    "learning_rate": 0.001,
    "validation_size": 0.3,
    "sample_rate": 44100,
    "n_fft": 4410,
    "n_mels": 128,
    "backend": "gloo"
}

# Create a PyTorch estimator
# env = {
#     'SAGEMAKER_REQUIREMENTS': './requirements.txt', # path relative to `source_dir` below.
# }
estimator = PyTorch(
    entry_point=entry_point_script,
    source_dir="./code",
    role=role,
    py_version="py38",
    framework_version="1.11.0",
    instance_count=2,
    instance_type="ml.c5.2xlarge",
    # instance_type="ml.g4dn.xlarge",
    hyperparameters=hyperparameters,
    sagemaker_session=sagemaker_session,
)

# Define the input data channels (S3 paths)
data_channels = {
    'train': f's3://{bucket}/{prefix}/train/'
}

# Start the training job
estimator.fit(data_channels)


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: pytorch-training-2024-04-21-01-21-28-714


2024-04-21 01:21:28 Starting - Starting the training job...
2024-04-21 01:21:44 Starting - Preparing the instances for training...
2024-04-21 01:22:19 Downloading - Downloading input data...
2024-04-21 01:22:39 Downloading - Downloading the training image......
2024-04-21 01:23:51 Training - Training image download completed. Training in progress.
2024-04-21 01:23:51 Uploading - Uploading generated training modelbash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2024-04-20 21:23:40,069 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2024-04-20 21:23:40,071 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-04-20 21:23:40,073 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-04-20 21:23:40,082 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2024-04-20 21:23:40,084 

UnexpectedStatusException: Error for Training job pytorch-training-2024-04-21-01-21-28-714: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
ExitCode 1
ErrorMessage "FileNotFoundError: [Errno 2] No such file or directory: '/opt/ml/input/data/train/metadata.csv'"
Command "/opt/conda/bin/python3.8 audio_resnet.py --backend gloo --batch_size 32 --epochs 10 --learning_rate 0.001 --n_fft 4410 --n_mels 128 --sample_rate 44100 --validation_size 0.3", exit code: 1

In [1]:
# test_inference.py
import torch
from my_model.code import inference  # This is the inference.py script you created
import librosa
import json

# Simulate SageMaker's model_fn
model_dir = './my_model'  # Replace with the path to your model directory
model = inference.model_fn(model_dir)

# Load a sample audio file using librosa
audio_path = '../data/train/1_10142.wav'  # Replace with the path to your audio file
signal, sr = librosa.load(audio_path, sr=None, mono=True)

# Create a JSON object with the signal and sample rate
input_json = json.dumps({'signal': signal.tolist(), 'sr': sr})

# Simulate SageMaker's input_fn with JSON content
input_data = inference.input_fn(input_json, 'application/json')


print(type(input_data))
print(input_data.shape)
# Simulate SageMaker's predict_fn
prediction = inference.predict_fn(input_data, model)

# Simulate SageMaker's output_fn
output = inference.output_fn(prediction, 'application/json')

# Print the output
print(output)


<class 'numpy.ndarray'>
(1, 3, 128, 250)
torch.Size([1, 3, 128, 250]) cuda
{'class': 1, 'confidence': 0.9617887735366821}


In [13]:
sr

44100

In [10]:
len(signal)

220500

In [14]:
print(torch.__version__)

2.2.2+cu118
